In [ ]:
import pandas as pd
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt
import onekey_algo.custom.components as okcomp
from onekey_algo import get_param_in_cwd

pd.set_option('display.float_format', lambda x: '{: .3f}'.format(x))
plt.rcParams['figure.dpi'] = 300
model_names = ['CEA', 'Pathology', 'Combined']
model_labels = ['Clincal model', 'Pathology model', 'Combined model']
# model_names = ['CEA', 'Pathology']
# 获取配置
task = get_param_in_cwd('task_column') or 'label'
bst_model = 'XGBoost'
clinic_model = 'ExtraTrees'
labelf = get_param_in_cwd('label_file')
group_info = get_param_in_cwd('dataset_column') or 'group'

# 读取label文件。
labels = [task]
label_data_ = pd.read_csv(labelf)
label_data_['ID'] = label_data_['ID'].astype(str)
label_data_ = label_data_[['ID', group_info, task]]
label_data_ = label_data_.dropna(axis=0)

ids = label_data_['ID']
print(label_data_.columns)
label_data = label_data_[['ID'] + labels]
label_data

# 训练集-Nomogram

In [ ]:
import pandas as pd

subset = 'train'
Clinic_results = pd.read_csv(f'clinic_sel.csv', header=0)[['ID', 'CEA']]
Clinic_results['ID'] = Clinic_results['ID'].astype(str)
Clinic_results['CEA'] = Clinic_results['CEA'] / np.max(Clinic_results['CEA'])
Clinic_results['CEA-0'] = 1 - Clinic_results['CEA']
Clinic_results = pd.merge(Clinic_results, label_data, on='ID', how='inner')
Path_results = pd.read_csv(f'./results/Path_{bst_model}_{subset}.csv', header=0)
Path_results['ID'] = Path_results['ID'].astype(str)
Path_results = pd.merge(Path_results, label_data, on='ID', how='inner')

ALL_results = pd.merge(Clinic_results, Path_results, on='ID', how='inner')
ALL_results.columns = ['ID', '-0', 'CEA', task, '-00', 'Pathology', '-ll']
# Clinical = pd.read_csv('clinic_sel.csv')[['ID', 'Smoking', 'Diabetes', 'Triglycerides', "eGFR", 'label']]
# Clinical['ID'] = Clinical['ID'].astype(str)
Clinic_results = pd.merge(ALL_results[['ID']], Clinic_results, on='ID', how='inner')

ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from onekey_algo.custom.components import metrics

mnames = ['SVM', 'KNN', 'ExtraTrees', 'XGBoost', 'MLP', 'LR', 'nb', 'AdaBoost']
models = okcomp.comp1.create_clf_model(mnames)
mnames = list(models.keys())

model = LogisticRegression(penalty='none')
# model = SVC(probability=True, random_state=0)
data_x = ALL_results[['CEA', 'Pathology']]
data_y = ALL_results[task]
model.fit(data_x, data_y)
results = model.predict_proba(data_x)
results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'{task}-0', f'{task}-1']).reset_index()
results.to_csv(f'./results/Nomo_{subset}.csv', index=False, header=True)
pd.DataFrame([metrics.analysis_pred_binary(ALL_results[task], results[f'{task}-1'])], 
                  columns=['acc', 'auc', '95%CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold'])

In [ ]:
pred_column = [f'{task}-0', f'{task}-1']
Nomo_results = pd.read_csv(f'./results/Nomo_{subset}.csv', header=0)
Nomo_results['ID'] = Nomo_results['ID'].astype(str)
Nomo_results = pd.merge(Nomo_results, label_data, on='ID', how='inner')
gt = [np.array(d) for d in [Clinic_results[labels], 
                            Path_results[labels],
                            Nomo_results[labels]]]
pred_train = [np.array(d) for d in [Clinic_results[['CEA-0', 'CEA']], 
                                    Path_results[pred_column],
                                    Nomo_results[pred_column]]]
okcomp.comp1.draw_roc(gt, pred_train, labels=model_labels, title=f"Model AUC")
plt.savefig(f'img/{subset}_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
metric = []
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score, use_youden=True)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Train"))
pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1','Threshold', 'Cohort'])

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test

Nomo_results.columns = ['ID', '-00000', 'Combined', '-llll']
ALL_results = pd.merge(ALL_results, Nomo_results, on='ID', how='inner')

delong = []

this_delong = []
delong_columns = []
for i, mni in enumerate(model_names):
    for _, mnj in enumerate(model_names[i+1:]):
        this_delong.append(delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0])
        delong_columns.append(f"{mni} Vs {mnj}")
this_delong.append('Train')
delong_columns.append('cohort')
delong.append(this_delong)
pd.DataFrame(this_delong, index=delong_columns).T

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([
          ALL_results[f'Pathology'],
          ALL_results[f'Combined']], 
         ALL_results[task], title=f'Model for DCA', labels=[f"{m} model" for m in model_names[1:]], y_min=-0.15)
plt.savefig(f'img/{subset}_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train[1:], n_bins=5, remap=True, #EX={'n_estimators': 10, 'max_depth': 4}, #smooth=True, # window_length=15, k=3,
                 y_test=gt[1:], model_names=model_labels[1:])
plt.savefig(f'img/{subset}_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer = []
hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred[:,1], bins=5) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.DataFrame(hosmer, columns=model_names)

In [ ]:
from onekey_algo.custom.components.comp1 import normalize_df
ALL_results = normalize_df(ALL_results, not_norm=['CEA', 'label', 'ID'], method='minmax')
ALL_results

In [ ]:
from onekey_algo.custom.components import nomogram
import shutil

ALL_results['CEA'] = ALL_results['CEA'] * 3
nomogram.risk_nomogram(ALL_results, result=task, columns=['CEA', 'Pathology'], 
                       width=8000, height=4000, x_range='0.05,0.25,0.50,0.75,0.95')

# 测试集-Nomogram

In [ ]:
import pandas as pd

subset = 'test'

Clinic_results = pd.read_csv(f'clinic_sel.csv', header=0)[['ID', model_names[0]]]
Clinic_results['ID'] = Clinic_results['ID'].astype(str)
Clinic_results['CEA'] = Clinic_results['CEA'] / np.max(Clinic_results['CEA'])
Clinic_results['CEA-0'] = 1 - Clinic_results['CEA']
Clinic_results = pd.merge(Clinic_results, label_data, on='ID', how='inner')
Path_results = pd.read_csv(f'./results/Path_{bst_model}_{subset}.csv', header=0)
Path_results['ID'] = Path_results['ID'].astype(str)
Path_results = pd.merge(Path_results, label_data, on='ID', how='inner')

ALL_results = pd.merge(Clinic_results, Path_results, on='ID', how='inner')
ALL_results.columns = ['ID', '-0', 'CEA', task, '-00', 'Pathology', '-ll']
# Clinical = pd.read_csv('clinic_sel.csv')[['ID', 'Smoking', 'Diabetes', 'Triglycerides', "eGFR", 'label']]
# Clinical['ID'] = Clinical['ID'].astype(str)
Clinic_results = pd.merge(ALL_results[['ID']], Clinic_results, on='ID', how='inner')

ALL_results = ALL_results.dropna(axis=1)
ALL_results

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components import metrics

model = LogisticRegression(random_state=0, penalty='none')
# model = SVC(probability=True, random_state=0)

# ALL_results = normalize_df(ALL_results, not_norm=['CEA', 'label', 'ID'], method='minmax')
data_x = ALL_results[['CEA', 'Pathology']]
data_y = ALL_results[task]
model.fit(data_x, data_y)
results = model.predict_proba(data_x)
results = pd.DataFrame(results, index=ALL_results['ID'], columns=[f'{task}-0', f'{task}-1']).reset_index()
results.to_csv(f'./results/Nomo_{subset}.csv', index=False, header=True)
pd.DataFrame([metrics.analysis_pred_binary(ALL_results[task], results[f'{task}-1'])], 
                  columns=['acc', 'auc', '95%CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold'])

In [ ]:
pred_column = [f'{task}-0', f'{task}-1']
Nomo_results = pd.read_csv(f'./results/Nomo_{subset}.csv', header=0)
Nomo_results['ID'] = Nomo_results['ID'].astype(str)
Nomo_results = pd.merge(Nomo_results, label_data, on='ID', how='inner')
gt = [np.array(d) for d in [Clinic_results[labels], 
                            Path_results[labels],
                            Nomo_results[labels]]]
pred_train = [np.array(d) for d in [Clinic_results[['CEA-0', 'CEA']], 
                                    Path_results[pred_column],
                                    Nomo_results[pred_column]]]
okcomp.comp1.draw_roc(gt, pred_train, labels=[f"{m}" for m in model_labels], title=f"Model AUC")
plt.savefig(f'img/{subset}_auc.svg')

In [ ]:
from onekey_algo.custom.components.metrics import analysis_pred_binary
for mname, y, score in zip(model_names, gt, pred_train):
    # 计算验证集指标
    acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
    ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
    metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Test"))
pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI',
                                          'Sensitivity', 'Specificity', 
                                          'PPV', 'NPV', 'Precision', 'Recall', 'F1', 'Threshold', 'Cohort'])

In [ ]:
from onekey_algo.custom.components.delong import delong_roc_test

Nomo_results.columns = ['ID', '-00000', 'Combined', '-llll']
ALL_results = pd.merge(ALL_results, Nomo_results, on='ID', how='inner')

this_delong = []
delong_columns = []
for i, mni in enumerate(model_names):
    for _, mnj in enumerate(model_names[i+1:]):
        this_delong.append(delong_roc_test(ALL_results[task], ALL_results[mni], ALL_results[mnj])[0][0])
        delong_columns.append(f"{mni} Vs {mnj}")
this_delong.append('Test')
delong_columns.append('cohort')
delong.append(this_delong)
pd.DataFrame(delong, columns=delong_columns)

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
plot_DCA([
          ALL_results[f'Pathology'],
          ALL_results[f'Combined']], 
         ALL_results[task], title=f'Model for DCA', labels=[f"{m} model" for m in model_names[1:]], y_min=-0.2)
plt.savefig(f'img/{subset}_dca.svg')

In [ ]:
from onekey_algo.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_train[1:], n_bins=5, remap=True, #EX={'n_estimators': 10, 'max_depth': 4}, #smooth=True, # window_length=15, k=3,
                 y_test=gt[1:], model_names=model_labels[1:])
plt.savefig(f'img/{subset}_cali.svg')

In [ ]:
from onekey_algo.custom.components import stats

hosmer.append([stats.hosmer_lemeshow_test(y_true, y_pred[:,1], bins=5) 
              for fn, y_true, y_pred in zip(model_names, gt, pred_train)])
pd.DataFrame(hosmer, columns=model_names)